# Guided Capstone Step 5. Modeling - Answer Key

**The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this guided capstone we are going to revisit many of the actions we took in the previous guided capstone. This gives you the opportunity to practice the code you wrote to solve the questions in step 4. 

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [2]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
os.listdir()

['updated_ski_data.csv',
 'GuidedCapstoneStep3-AnswerKey.ipynb',
 'GuidedCapstoneStep4-AnswerKey.ipynb',
 '.DS_Store',
 'GuidedCapstoneStep5-AnswerKey.ipynb',
 'GuidedCapstoneStep5_draft.ipynb',
 'models',
 'GuidedCapstoneStep2-AnswerKey.ipynb',
 'AnnotatedBoxplot.png',
 'figures',
 'GuidedCapstoneStep3.ipynb',
 'Cluster heatmaps.ipynb',
 '.ipynb_checkpoints',
 'GuidedCapstoneStep4.ipynb',
 'data',
 'GuidedCapstoneStep2.ipynb']

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [3]:
file="data/step3_output.csv"
df=pd.read_csv(file)
df.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,250,1,0,0,2,2,...,1610.0,113,150.0,60.0,669.0,65.0,85.0,150.0,550,1
1,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0,0,0,0,...,640.0,60,45.0,44.0,350.0,47.0,53.0,90.0,0,1
2,Hilltop Ski Area,Alaska,2090,294,1796,0,0,0,0,0,...,30.0,30,150.0,36.0,69.0,30.0,34.0,152.0,30,1
3,Arizona Snowbowl,Arizona,11500,2300,9200,0,0,1,0,2,...,777.0,104,122.0,81.0,260.0,89.0,89.0,122.0,0,0
4,Sunrise Park Resort,Arizona,11100,1800,9200,0,0,0,1,2,...,800.0,80,115.0,49.0,250.0,74.0,78.0,104.0,80,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

In [4]:
df = pd.concat([df, pd.get_dummies(df['state'])], axis=1).drop(['state'], axis =1)
print(df.shape)
df.head()

(330, 61)


,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,250,1,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,1200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,1796,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,9200,0,0,1,0,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,9200,0,0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [5]:
#scale data
from sklearn import preprocessing
X=df.drop(['Name','AdultWeekend'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [6]:
from sklearn.model_selection import train_test_split
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn fit the model on your training dataset.</font>**

#### Model 1

In [7]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [30]:
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed.

In [31]:
explained_variance_score(y_test, y_pred)

0.9279139290249572

In [92]:
mean_absolute_error(y_test, y_pred)

5.3461366383814335

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [34]:
lm.intercept_

64.13132886543396

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for `AdultWeekend` resulting value. Also, because we took the time to scale our x values in the training data we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [28]:
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
New York,9.379699e+12
Michigan,8.851889e+12
Colorado,7.799026e+12
California,7.632074e+12
Pennsylvania,7.283006e+12
Wisconsin,6.715502e+12
New Hampshire,6.715502e+12
Vermont,6.512603e+12
Minnesota,6.301751e+12
Utah,6.082120e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [89]:
file="data/step3_output.csv"
df=pd.read_csv(file)
X=df.drop(['AdultWeekend','Name','state'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(explained_variance_score(y_test, y_pred))
mean_absolute_error(y_test, y_pred)

0.9223038219833387


5.5432253630150585

In [90]:
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,19.940382
summit_elev,13.955579
base_elev,10.725078
vertical_drop,4.949809
quad,1.530338
Runs,1.525236
averageSnowfall,1.509100
triple,1.426216
surface,1.292115
daysOpenLastYear,0.960395


**<font color='teal'> When reviewing our new model coefficients we see `summit_elev` is now in the number two spot, this also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [120]:
file="data/step3_output.csv"
df=pd.read_csv(file)
X=df.drop(['Name','AdultWeekend','state','summit_elev','base_elev'], axis=1)
y=df.AdultWeekend
scaler = preprocessing.StandardScaler().fit(X)
X_scaled=scaler.transform(X) 
y=y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(explained_variance_score(y_test, y_pred))
mean_absolute_error(y_test, y_pred)

0.9239026961737301


5.531090105098711

In [121]:
coeff_df = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,19.893397
averageSnowfall,1.982708
Runs,1.665804
quad,1.578662
triple,1.380822
vertical_drop,1.291189
surface,1.266572
daysOpenLastYear,0.829641
fastQuads,0.771395
clusters,0.766623


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell and you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.928 | 5.35 |-|
| Model 2. | 0.922 | 5.54 |'state'|
| Model 3. | 0.924 | 5.53 |'state','summit_elev','base_elev'|

Model Selection: